In [1]:
###### import numpy as np
import pandas as pd
import os
import time
import schedule
import joblib
import datetime
import multiprocessing as mp
from get_strategy_meta import get_all_strategy_meta
from query_api import get_path_dict, gen_connection, load_item_by_key, query_mysql
from get_signal_mongo import write_signal_mongo, get_signal_mongo
from get_signal_mysql import write_signal, get_signal
from mongo_functions import get_portfolio_db
import shutil

Using TensorFlow backend.


In [2]:
strategy_meta_df = get_all_strategy_meta()

In [3]:
sdf = strategy_meta_df.loc[~strategy_meta_df["Backtest Status"].isin(["Abandon"])]
sdf["TF"] = sdf["TimeFrame"].apply(lambda x: int(x.split("_")[1]))
sdf = sdf.sort_values("TF")



In [72]:
strat = "1301"
strategy_meta = sdf.loc[sdf["Strategy"] == strat].to_dict(orient = "records")[0]
PD = get_path_dict(strategy_meta)

file_list = []
for k,v in PD.items():
    for k2, v2 in v.items():
        file_list.extend(v2)
new_file_list = [x.replace("DL_Output", "DLBatchOne/Accepted_Windows") for x in file_list]

In [73]:
new_file_list

['/media/workstation/Storage/DLBatchOne/Accepted_Windows/Z/Z007B_D0H0M5_B/ASSETS_MODELS/EURJPY_1547237855/scaler.pkl',
 '/media/workstation/Storage/DLBatchOne/Accepted_Windows/Z/Z007B_D0H0M5_B/ASSETS_MODELS/EURJPY_1547237855/DATA_PARAMS.pkl',
 '/media/workstation/Storage/DLBatchOne/Accepted_Windows/Z/Z007B_D0H0M5_B/ASSETS_MODELS/EURJPY_1547237855/MODEL_PARAMS.pkl',
 '/media/workstation/Storage/DLBatchOne/Accepted_Windows/Z/Z007B_D0H0M5_B/ASSETS_MODELS/EURJPY_1547237855/DL-0036-0.5318-0.7817-0.1671-0.0836.model',
 '/media/workstation/Storage/DLBatchOne/Accepted_Windows/Z/Z007B_D0H0M5_B/ASSETS_MODELS/NZDUSD_1547328136/scaler.pkl',
 '/media/workstation/Storage/DLBatchOne/Accepted_Windows/Z/Z007B_D0H0M5_B/ASSETS_MODELS/NZDUSD_1547328136/DATA_PARAMS.pkl',
 '/media/workstation/Storage/DLBatchOne/Accepted_Windows/Z/Z007B_D0H0M5_B/ASSETS_MODELS/NZDUSD_1547328136/MODEL_PARAMS.pkl',
 '/media/workstation/Storage/DLBatchOne/Accepted_Windows/Z/Z007B_D0H0M5_B/ASSETS_MODELS/NZDUSD_1547328136/DL-0066-

In [74]:
file_list

['/media/workstation/Storage/DL_Output/Z/Z007B_D0H0M5_B/ASSETS_MODELS/EURJPY_1547237855/scaler.pkl',
 '/media/workstation/Storage/DL_Output/Z/Z007B_D0H0M5_B/ASSETS_MODELS/EURJPY_1547237855/DATA_PARAMS.pkl',
 '/media/workstation/Storage/DL_Output/Z/Z007B_D0H0M5_B/ASSETS_MODELS/EURJPY_1547237855/MODEL_PARAMS.pkl',
 '/media/workstation/Storage/DL_Output/Z/Z007B_D0H0M5_B/ASSETS_MODELS/EURJPY_1547237855/DL-0036-0.5318-0.7817-0.1671-0.0836.model',
 '/media/workstation/Storage/DL_Output/Z/Z007B_D0H0M5_B/ASSETS_MODELS/NZDUSD_1547328136/scaler.pkl',
 '/media/workstation/Storage/DL_Output/Z/Z007B_D0H0M5_B/ASSETS_MODELS/NZDUSD_1547328136/DATA_PARAMS.pkl',
 '/media/workstation/Storage/DL_Output/Z/Z007B_D0H0M5_B/ASSETS_MODELS/NZDUSD_1547328136/MODEL_PARAMS.pkl',
 '/media/workstation/Storage/DL_Output/Z/Z007B_D0H0M5_B/ASSETS_MODELS/NZDUSD_1547328136/DL-0066-0.6503-0.7281-0.2012-0.1517.model',
 '/media/workstation/Storage/DL_Output/Z/Z007B_D0H0M5_B/ASSETS_MODELS/XAGUSD_1547357323/scaler.pkl',
 '/medi

In [75]:
for x, y in zip(file_list, new_file_list):
    if not os.path.exists(os.path.dirname(y)):
        try:
            os.makedirs(os.path.dirname(y))
        except OSError as exc: # Guard against race condition
            raise


    shutil.copy(x, y)